In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import requests as req
import json
import pymysql
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date
from datetime import datetime
from warnings import filterwarnings
filterwarnings('ignore', category=pymysql.Warning)

In [2]:
engine = create_engine('sqlite:///hawaii.sqlite')
conn = engine.connect()

In [3]:
Base = declarative_base()

class Stations(Base):
    __tablename__= 'stations'
        
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    

In [4]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [5]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [6]:
Base = declarative_base()

class Measures(Base):
    __tablename__= 'measures'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

In [7]:
Base.metadata.create_all(engine)

In [8]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [9]:
file_name = os.path.join("clean_hawaii_measurements.csv")
hawaii_pd = pd.read_csv(file_name)
del hawaii_pd['Unnamed: 0']
hawaii_pd.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [10]:
hawaii_pd['date'] = pd.to_datetime(hawaii_pd['date'])

In [11]:
file_name2 = os.path.join("clean_hawaii_stations.csv")
stations_pd = pd.read_csv(file_name2)
del stations_pd['Unnamed: 0']
stations_pd

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [12]:
engine.table_names()

['measures', 'stations']

In [13]:
hawaii_dict = hawaii_pd.to_dict(orient='records')

In [14]:
conn.execute(Measures.__table__.insert(), hawaii_dict)

In [15]:
engine.execute('SELECT * FROM Measures LIMIT 5').fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [16]:
station_dict = stations_pd.to_dict(orient='records')

In [17]:
conn.execute(Stations.__table__.insert(), station_dict)

In [18]:
engine.execute('SELECT * FROM Stations LIMIT 5').fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [19]:
#measures_df.plot(kind='scatter', x='date', y='prcp', marker='o')
#measures_df['date'] = pd.to_datetime(measures_df['date'], format = '%Y-%m-%d')